In [13]:
from ortools.linear_solver import pywraplp

In [14]:
# Crea el solucionador lineal con el backend GLOP
solver = pywraplp.Solver.CreateSolver("GLOP")
if not solver:
    print("No se pudo crear el solucionador GLOP")

numvar continua  
intvar  entera  
solver.IntVar(0, 1, "bin_x") 

In [15]:
x_var = solver.NumVar(0, 1, "x") #numvar crea una variable continua 
y_var = solver.NumVar(0, 2, "y")

para crear una variable sin limtie se usa solver.infinity()

In [16]:
print("Number of variables =", solver.NumVariables())


Number of variables = 2


In [5]:
#3x+2y<=10
inf=solver.infinity()
restriccion=solver.Constraint(inf,10,"r1")
restriccion.SetCoefficient(x_var,3)
restriccion.SetCoefficient(y_var,2)

In [17]:
infinity = solver.infinity()
# Create a linear constraint, x + y <= 2.
constraint = solver.Constraint(-infinity, 2, "ct")
constraint.SetCoefficient(x_var, 1)
constraint.SetCoefficient(y_var, 1)

print("Number of constraints =", solver.NumConstraints())

Number of constraints = 1


In [18]:
def print_constraint_expression(solver, constraint):
    """Reconstruye y muestra la expresión matemática de una restricción."""
    
    # 1. Obtener los límites de la restricción
    lower_bound = constraint.Lb()
    upper_bound = constraint.Ub()
    name = constraint.name()
    
    # 2. Reconstruir el lado izquierdo (Left-Hand Side, LHS)
    lhs_parts = []
    
    # Iterar sobre todas las variables que se agregaron al solver
    for var in solver.variables():
        # Obtener el coeficiente de esta variable para la restricción
        coeff = constraint.GetCoefficient(var)
        
        if coeff != 0:
            # Formatea el término (ej: "3*x" o "+ 5*y")
            if len(lhs_parts) == 0:
                # El primer término no lleva un '+' al inicio
                term = f"{coeff}*{var.name()}"
            else:
                # Los términos subsiguientes llevan su signo
                sign = "+" if coeff > 0 else ""
                term = f"{sign} {coeff}*{var.name()}"
            
            lhs_parts.append(term)

    # Si no hay variables en la restricción (debería ser raro)
    if not lhs_parts:
        lhs_str = "0"
    else:
        lhs_str = " ".join(lhs_parts)

    # 3. Formatear la salida completa
    
    # Usar .infinity() para una salida limpia
    INF = solver.infinity()
    
    if lower_bound == upper_bound:
        # Caso de igualdad (ej: LHS = 10)
        relation_str = f"= {lower_bound}"
    elif lower_bound > -INF and upper_bound == INF:
        # Caso de límite inferior (ej: LHS >= 10)
        relation_str = f">= {lower_bound}"
    elif lower_bound == -INF and upper_bound < INF:
        # Caso de límite superior (ej: LHS <= 10)
        relation_str = f"<= {upper_bound}"
    elif lower_bound > -INF and upper_bound < INF:
        # Caso de rango (ej: 5 <= LHS <= 10)
        relation_str = f">= {lower_bound} y LHS <= {upper_bound}"
    else:
        # Cualquier otro caso (no acotado)
        relation_str = "no acotada"
        
    print(f"Restricción '{name}': {lhs_str} {relation_str}")

In [19]:
for c in solver.constraints():
    print_constraint_expression(solver, c)

Restricción 'ct': 1.0*x + 1.0*y <= 2.0


Constraint para restriccion Objective para la f.o

In [20]:
#maximizar 3*x + y

In [21]:
objetivo=solver.Objective()
objetivo.SetCoefficient(x_var,3)
objetivo.SetCoefficient(y_var,1)
objetivo.SetMaximization()

In [22]:
solucion = solver.Solve()
if solucion == pywraplp.Solver.OPTIMAL:
    print("Solución óptima encontrada.")
    print(f"Valor óptimo de la función objetivo: {solver.Objective().Value()}")
    print(f"Valor de x: {x_var.SolutionValue()}")
    print(f"Valor de y: {y_var.SolutionValue()}")
else:
    print("El problema no tiene solución óptima (ej. es infactible o no acotado).")

Solución óptima encontrada.
Valor óptimo de la función objetivo: 4.0
Valor de x: 1.0
Valor de y: 1.0


In [12]:
# Exporta el modelo a un archivo .lp
# El formato LP (Linear Programming) es legible por humanos
lp_model_string = solver.ExportModelAsLpFormat(False) # False para no exportar comentarios
with open("mi_modelo_grande.lp", "w") as f:
    f.write(lp_model_string)